# AgentCore Memory를 사용한 Strands Agent (장기 메모리) - MemoryManager

## 소개

이 튜토리얼은 **MemoryManager**와 **MemorySessionManager**를 사용하여 hook을 통해 AgentCore Memory와 통합된 Strands agent를 사용하여 **지능형 고객 지원 agent**를 구축하는 방법을 보여줍니다. 고객 상호작용 기록을 위한 장기 메모리에 중점을 두고, 구매 세부 정보를 기억하며, 이전 대화 및 사용자 선호도를 기반으로 개인화된 지원을 제공합니다.

**참고: 이것은 구현을 위해 MemoryManager 및 MemorySessionManager를 사용하는 샘플입니다.**

### 튜토리얼 세부 정보

| 정보         | 세부 사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 장기 대화형                                                         |
| Agent 유형          | 고객 지원                                                                 |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Haiku 4.5                                                      |
| 튜토리얼 구성 요소 | AgentCore Semantic 및 User Preferences Memory 추출, Memory 저장 및 검색을 위한 Hook              |
| 예제 복잡도  | 중급                                                                     |

다음을 배우게 됩니다:
- MemoryManager를 사용하여 장기 전략으로 AgentCore Memory 설정
- MemorySessionManager를 사용한 자동 저장 및 검색을 위한 메모리 hook 생성
- 지속적인 메모리를 가진 고객 지원 agent 구축
- 이전 상호작용의 컨텍스트를 사용하여 고객 문제 처리

### 시나리오 컨텍스트
이 예제에서는 **고객 지원 사용 사례**를 구축합니다. Agent는 주문 기록, 선호도 및 이전 문제를 포함한 고객 컨텍스트를 기억하여 보다 개인화되고 효과적인 지원을 가능하게 합니다. 고객과의 대화는 메모리 hook을 사용하여 자동으로 저장되므로 중요한 세부 정보가 손실되지 않습니다. semantic 및 User Preference와 같은 여러 메모리 전략을 사용함으로써 agent는 광범위한 관련 정보를 캡처할 수 있습니다. 이 설정을 통해 agent는 고객의 기록 및 선호도를 완전히 인식하면서 문제를 해결할 수 있습니다. 또한 agent는 웹 검색 기능과 통합되어 필요에 따라 최신 제품 정보 및 문제 해결 가이드를 쉽게 제공할 수 있습니다.

## 아키텍처

<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>


## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
- Python 3.10+
- Amazon Bedrock AgentCore Memory 권한이 있는 AWS 자격 증명
- MemoryManager 지원이 포함된 Amazon Bedrock AgentCore SDK

## 단계 1: 종속성 설치 및 설정
이 노트북이 작동하도록 필요한 모든 라이브러리를 가져오고 클라이언트를 정의하는 것으로 시작하겠습니다.

In [1]:
!pip install -qr requirements.txt

In [2]:
import logging
import json
from typing import Dict, List
from datetime import datetime
from botocore.exceptions import ClientError

# 로깅 설정
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("customer-support")

# Strands Agent 관련 모듈 import
from strands import Agent, tool
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent
from ddgs import DDGS

# AgentCore Memory 관리 모듈 import
from bedrock_agentcore_starter_toolkit.operations.memory.manager import Memory, MemoryManager
from bedrock_agentcore_starter_toolkit.operations.memory.models.strategies import (
    SemanticStrategy, SummaryStrategy, CustomSemanticStrategy, CustomSummaryStrategy, CustomUserPreferenceStrategy,
    ExtractionConfig, ConsolidationConfig
)
from bedrock_agentcore.memory.constants import BlobMessage, ConversationalMessage, MessageRole, RetrievalConfig
from bedrock_agentcore.memory.session import Actor, MemorySession, MemorySessionManager
from bedrock_agentcore.memory.models import MemoryRecord

# 메시지 역할 상수 정의
USER = MessageRole.USER
ASSISTANT = MessageRole.ASSISTANT

In [3]:
# 설정 - 올바른 값으로 변경 필요
REGION = "us-east-1"  
CUSTOMER_ID = "customer_001"
SESSION_ID = f"support_{datetime.now().strftime('%Y%m%d%H%M%S')}"

# IAM role 생성을 위한 boto3 import
import boto3
import json as json_module
from botocore.exceptions import ClientError

## 단계 1.1: 커스텀 Memory 전략을 위한 IAM Role 생성

커스텀 메모리 전략은 AgentCore Memory가 추출 및 통합을 위해 Bedrock model을 호출할 수 있도록 하는 실행 role이 필요합니다. 이 role은 `CustomSemanticStrategy` 또는 `CustomUserPreferenceStrategy`를 사용할 때 필요합니다.

In [ ]:
# AgentCore Memory 커스텀 전략을 위한 IAM role 생성
def create_memory_execution_role():
    """필요한 권한을 가진 AgentCore Memory 커스텀 전략용 IAM role 생성"""
    iam_client = boto3.client('iam', region_name=REGION)
    
    # 현재 AWS 계정 ID 가져오기
    sts_client = boto3.client('sts', region_name=REGION)
    account_id = sts_client.get_caller_identity()['Account']
    
    role_name = "AgentCoreMemoryExecutionRole"
    role_arn = f"arn:aws:iam::{account_id}:role/{role_name}"
    
    # AgentCore Memory 서비스를 위한 신뢰 정책
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "",
                "Effect": "Allow",
                "Principal": {
                    "Service": [
                        "bedrock-agentcore.amazonaws.com"
                    ]
                },
                "Action": "sts:AssumeRole",
                "Condition": {
                    "StringEquals": {
                        "aws:SourceAccount": account_id
                    },
                    "ArnLike": {
                        "aws:SourceArn": f"arn:aws:bedrock-agentcore:{REGION}:{account_id}:*"
                    }
                }
            }
        ]
    }
    
    # Bedrock model 호출을 위한 권한 정책
    permissions_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock:InvokeModel",
                    "bedrock:InvokeModelWithResponseStream"
                ],
                "Resource": [
                    "arn:aws:bedrock:*::foundation-model/*",
                    "arn:aws:bedrock:*:*:inference-profile/*"
                ],
                "Condition": {
                    "StringEquals": {
                        "aws:ResourceAccount": account_id
                    }
                }
            }
        ]
    }
    
    try:
        # role이 이미 존재하는지 확인
        try:
            existing_role = iam_client.get_role(RoleName=role_name)
            logger.info(f"✅ IAM role already exists: {role_arn}")
            return role_arn
        except ClientError as e:
            if e.response['Error']['Code'] != 'NoSuchEntity':
                raise
        
        # role 생성
        logger.info(f"Creating IAM role: {role_name}")
        iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json_module.dumps(trust_policy),
            Description="Execution role for AgentCore Memory custom strategies",
            Tags=[
                {
                    'Key': 'Purpose',
                    'Value': 'AgentCoreMemory'
                }
            ]
        )
        
        # 권한 정책 연결
        policy_name = "AgentCoreMemoryBedrockAccess"
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=policy_name,
            PolicyDocument=json_module.dumps(permissions_policy)
        )
        
        logger.info(f"✅ Successfully created IAM role: {role_arn}")
        logger.info(f"   - Trust policy: AgentCore Memory service can assume this role")
        logger.info(f"   - Permissions: bedrock:InvokeModel and bedrock:InvokeModelWithResponseStream")
        
        return role_arn
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'AccessDenied':
            logger.error("❌ Access denied creating IAM role. Please ensure you have IAM permissions:")
            logger.error("   - iam:CreateRole")
            logger.error("   - iam:PutRolePolicy")
            logger.error("   - iam:GetRole")
        else:
            logger.error(f"❌ Failed to create IAM role: {e}")
        raise
    except Exception as e:
        logger.error(f"❌ Unexpected error creating IAM role: {e}")
        raise

# execution role 생성
try:
    MEMORY_EXECUTION_ROLE_ARN = create_memory_execution_role()
    logger.info(f"✅ Memory execution role ready: {MEMORY_EXECUTION_ROLE_ARN}")
except Exception as e:
    logger.error(f"❌ Failed to create memory execution role: {e}")
    # 데모 목적으로, 생성 실패 시 수동으로 role ARN 설정 가능
    # MEMORY_EXECUTION_ROLE_ARN = "arn:aws:iam::YOUR_ACCOUNT_ID:role/YourExistingRole"
    raise

## 단계 2: 고객 지원을 위한 Memory 리소스 생성

고객 지원을 위해 여러 메모리 전략을 사용합니다:
- **CustomUserPreferenceStrategy**: 고객 선호도 및 행동 캡처
- **CustomSemanticStrategy**: 주문 사실 및 제품 정보 저장

**중요**: 커스텀 전략은 AgentCore Memory가 Bedrock model을 호출할 수 있도록 하는 IAM 실행 role이 필요합니다. 위의 단계 1.1에서 이 role을 생성했습니다.

In [ ]:
# Memory Manager 초기화
memory_manager = MemoryManager(region_name=REGION)
memory_name = "CustomerSupportLongTermMemory"

# 기본 memory manager 초기화 및 연결 테스트
logger.info(f"✅ MemoryManager initialized for region: {REGION}")
logger.info(f"Memory manager type: {type(memory_manager)}")

# 타입이 지정된 클래스를 사용하여 메모리 전략 정의
strategies = [
    CustomUserPreferenceStrategy(
        name="CustomerPreferences",
        description="Captures customer preferences and behavior",
        extraction_config=ExtractionConfig(
            append_to_prompt="Extract customer preferences and behavior patterns",
            model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0"
        ),
        consolidation_config=ConsolidationConfig(
            append_to_prompt="Consolidate customer preferences",
            model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0"
        ),
        namespaces=["support/customer/{actorId}/preferences/"]
    ),
    CustomSemanticStrategy(
        name="CustomerSupportSemantic",
        description="Stores facts from conversations",
        extraction_config=ExtractionConfig(
            append_to_prompt="Extract factual information from customer support conversations",
            model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0"
        ),
        consolidation_config=ConsolidationConfig(
            append_to_prompt="Consolidate semantic insights from support interactions",
            model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0"
        ),
        namespaces=["support/customer/{actorId}/semantic/"]
    )
]

# 전략 설정 검증
logger.info(f"✅ Configured {len(strategies)} memory strategies:")
for i, strategy in enumerate(strategies, 1):
    logger.info(f"  {i}. {strategy.name} ({type(strategy).__name__})")
    logger.info(f"     Description: {strategy.description}")
    logger.info(f"     Namespaces: {strategy.namespaces}")
    if hasattr(strategy, 'extraction_config') and strategy.extraction_config:
        logger.info(f"     Extraction Model: {strategy.extraction_config.model_id}")
    if hasattr(strategy, 'consolidation_config') and strategy.consolidation_config:
        logger.info(f"     Consolidation Model: {strategy.consolidation_config.model_id}"),

# MemoryManager를 사용하여 메모리 리소스 생성
logger.info(f"Creating memory '{memory_name}' with {len(strategies)} strategies...")

try:
    memory = memory_manager.get_or_create_memory(
        name=memory_name,
        strategies=strategies,         # 타입이 지정된 전략 객체 전달
        description="Memory for customer support agent",
        event_expiry_days=90,          # 메모리는 90일 후 만료
        memory_execution_role_arn=MEMORY_EXECUTION_ROLE_ARN,  # Custom 전략에 필요
    )
    memory_id = memory.id
    logger.info(f"✅ Successfully created/retrieved memory with MemoryManager:")
    logger.info(f"   Memory ID: {memory_id}")
    logger.info(f"   Memory Name: {memory.name}")
    logger.info(f"   Memory Status: {memory.status}")
    
except Exception as e:
    # 향상된 오류 보고와 함께 메모리 생성 중 오류 처리
    logger.error(f"❌ Memory creation failed: {e}")
    logger.error(f"Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()
    
    # 오류 발생 시 정리 - 부분적으로 생성된 메모리 삭제
    if 'memory_id' in locals():
        try:
            logger.info(f"Attempting cleanup of partially created memory: {memory_id}")
            memory_manager.delete_memory(memory_id)
            logger.info(f"✅ Successfully cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"❌ Failed to clean up memory: {cleanup_error}")
    
    # 원래 예외 다시 발생
    raise

In [ ]:
# memory manager 기본 기능 테스트
try:
    # 기존 메모리 목록 조회 테스트
    existing_memories = memory_manager.list_memories()
    logger.info(f"✅ Memory manager connection successful. Found {len(existing_memories)} existing memories")
    
    # 기존 테스트 메모리 정리
    for mem in existing_memories:
        if mem.name and mem.name.startswith("CustomerSupportLongTermMemory"):
            logger.info(f"Cleaning up existing test memory: {mem.id}")
            memory_manager.delete_memory(mem.id)
            
except Exception as e:
    logger.error(f"❌ Memory manager test failed: {e}")
    raise

메모리에 할당한 전략이 포함되어 있는지 확인해 보겠습니다

In [ ]:
# MemoryManager를 사용하여 메모리 정보 표시
print(f"Memory ID: {memory.id}")
print(f"Memory Name: {memory.name}")
print(f"Memory Description: {memory.description}")
print(f"Memory Status: {memory.status}")
print(f"Number of strategies: {len(strategies)}")
for i, strategy in enumerate(strategies, 1):
    print(f"  {i}. {strategy.name}: {strategy.description}")

## 단계 3: Agent Tool 생성

In [ ]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

@tool
def web_search(query: str, max_results: int = 3) -> str:
    """제품 정보, 문제 해결 가이드 또는 지원 문서를 웹에서 검색합니다.
    
    Args:
        query: 제품 정보 또는 문제 해결을 위한 검색 쿼리
        max_results: 반환할 최대 결과 수
    
    Returns:
        제목과 요약이 포함된 검색 결과
    """
    try:
        results = DDGS().text(query, region="us-en", max_results=max_results)
        if not results:
            return "No search results found."
        
        formatted_results = []
        for i, result in enumerate(results, 1):
            formatted_results.append(f"{i}. {result.get('title', 'No title')}\n   {result.get('body', 'No description')}")
        
        return "\n".join(formatted_results)
    except RatelimitException:
        return "Rate limit reached: Please try again after a short delay."
    except DDGSException as d:
        return f"Search Error: {d}"
    except Exception as e:
        return f"Search error: {str(e)}"

logger.info("✅ Web search tool ready")

@tool
def check_order_status(order_number: str) -> str:
    """고객 주문 상태를 확인합니다.
    
    Args:
        order_number: 확인할 주문 번호
    
    Returns:
        주문 상태 정보
    """
    # 주문 조회 시뮬레이션
    mock_orders = {
        "123456": "iPhone 15 Pro - Delivered on June 5, 2025",
        "654321": "Sennheiser Headphones - Delivered on June 25, 2025, 1-year warranty active",
        "789012": "Samsung Galaxy S23 - In transit, expected delivery on July 1, 2025",
    }
    
    return mock_orders.get(order_number, f"Order {order_number} not found. Please verify the order number.")

logger.info("✅ Check Order Status tool ready")

## 단계 4: Session Manager 초기화

**새로운 기능: 이 섹션에서는 세션 기반 Memory 작업을 위한 MemorySessionManager를 소개합니다.**

In [ ]:
# session memory manager 초기화
session_manager: MemorySessionManager = MemorySessionManager(memory_id=memory.id, region_name=REGION)

# 특정 고객을 위한 메모리 세션 생성
customer_session: MemorySession = session_manager.create_memory_session(
    actor_id=CUSTOMER_ID, 
    session_id=SESSION_ID
)

logger.info(f"✅ Session manager initialized for memory: {memory.id}")
logger.info(f"✅ Customer session created for actor: {CUSTOMER_ID}")
logger.info(f"   Session type: {type(customer_session)}")
logger.info(f"   Actor object: {customer_session.get_actor()}")

## 단계 5: 고객 지원을 위한 Memory Hook Provider 생성
Hook은 agent의 실행 수명 주기의 특정 지점에서 실행되는 특수 함수입니다. 우리의 커스텀 hook provider는 다음을 통해 고객 지원 컨텍스트를 자동으로 관리합니다:
- 세션 기반 메서드를 사용하여 각 응답 후 **지원 상호작용 저장**
- 새 쿼리를 처리할 때 이전 주문 및 선호도에서 **관련 컨텍스트 검색 및 주입**.


In [ ]:
class CustomerSupportMemoryHooks(HookProvider):
    """고객 지원 agent를 위한 Memory hook - MemorySession으로 향상됨"""
    
    def __init__(self, customer_session: MemorySession):
        # MemorySession을 직접 받음
        self.customer_session = customer_session
        
        # 다양한 메모리 타입에 대한 검색 설정 정의
        self.retrieval_config = {
            "support/customer/{actorId}/preferences/": RetrievalConfig(top_k=3, relevance_score=0.3),
            "support/customer/{actorId}/semantic/": RetrievalConfig(top_k=5, relevance_score=0.2)
        }
    
    def retrieve_customer_context(self, event: MessageAddedEvent):
        """MemorySession을 사용하여 지원 쿼리 처리 전 고객 컨텍스트 검색"""
        messages = event.agent.messages
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_query = messages[-1]["content"][0]["text"]
            
            try:
                # 컨텍스트 검색을 위해 MemorySession 사용
                relevant_memories = []
                
                # MemorySession을 사용하여 다양한 메모리 namespace 검색
                for namespace_template, config in self.retrieval_config.items():
                    # namespace 템플릿을 세션의 실제 actor ID로 해석
                    resolved_namespace = namespace_template.format(
                        actorId=self.customer_session._actor_id
                    )
                    
                    # MemorySession API 사용 (actor_id/session_id 전달 불필요)
                    memories = self.customer_session.search_long_term_memories(
                        query=user_query,
                        namespace_prefix=resolved_namespace,
                        top_k=config.top_k
                    )
                    print(memories)
                    # 관련성 점수로 필터링
                    filtered_memories = [
                        memory for memory in memories
                        if memory.get("score", 0) >= config.relevance_score
                    ]
                    
                    relevant_memories.extend(filtered_memories)
                    logger.info(f"Found {len(filtered_memories)} relevant memories in {resolved_namespace}")
                
                # 메모리가 발견되면 agent의 system prompt에 컨텍스트 주입
                if relevant_memories:
                    context_text = self._format_context(relevant_memories)
                    original_prompt = event.agent.system_prompt
                    enhanced_prompt = f"{original_prompt}\n\nCustomer Context:\n{context_text}"
                    event.agent.system_prompt = enhanced_prompt
                    logger.info(f"✅ Injected {len(relevant_memories)} memories into agent context")
                    
            except Exception as e:
                logger.error(f"Failed to retrieve customer context: {e}")
    
    def _format_context(self, memories: List[MemoryRecord]) -> str:
        """agent 컨텍스트를 위해 검색된 메모리 포맷팅"""
        context_lines = []
        for i, memory in enumerate(memories[:5], 1):  # 상위 5개로 제한
            content = memory.get('content', {}).get('text', 'No content available')
            score = memory.get('score', 0)
            context_lines.append(f"{i}. (Score: {score:.2f}) {content[:200]}...")
        
        return "\n".join(context_lines)
    
    def save_support_interaction(self, event: AfterInvocationEvent):
        """MemorySession을 사용하여 지원 상호작용 저장 (더 깔끔한 API)"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # 마지막 고객 쿼리와 agent 응답 가져오기
                customer_query = None
                agent_response = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not agent_response:
                        agent_response = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not customer_query and "toolResult" not in msg["content"][0]:
                        customer_query = msg["content"][0]["text"]
                        break
                
                if customer_query and agent_response:
                    # MemorySession 사용 (actor_id/session_id 전달 불필요)
                    interaction_messages = [
                        ConversationalMessage(customer_query, USER),
                        ConversationalMessage(agent_response, ASSISTANT)
                    ]
                    
                    result = self.customer_session.add_turns(interaction_messages)
                    logger.info(f"✅ Saved interaction using MemorySession - Event ID: {result['eventId']}")
                    
        except Exception as e:
            logger.error(f"Failed to save support interaction: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        """고객 지원 메모리 hook 등록"""
        registry.add_callback(MessageAddedEvent, self.retrieve_customer_context)  # 다시 추가됨!
        registry.add_callback(AfterInvocationEvent, self.save_support_interaction)
        logger.info("✅ Customer support memory hooks registered with MemorySession")
print('Executed!')

### 단계 6: 고객 지원 Agent 생성

In [ ]:
# MemorySession을 사용하여 메모리 hook 생성
support_hooks = CustomerSupportMemoryHooks(customer_session)

# 고객 지원 agent 생성
support_agent = Agent(
    hooks=[support_hooks],
    model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    tools=[web_search, check_order_status],
    system_prompt="""You are a helpful customer support agent with access to customer history and order information. 
    
    Your role:
    - Help customers with their orders, returns, and product issues
    - Use customer context to provide personalized support
    - Search for product information when needed
    - Be empathetic and solution-focused
    - Reference previous orders and preferences when relevant
    
    Always be professional, helpful, and aim to resolve customer issues efficiently."""
)

print("✅ Customer support agent created with MemorySession integration")

### 단계 7: 고객 기록 시드

메모리 기능을 시연하기 위해 이전 고객 상호작용을 추가해 보겠습니다.

**참고: 이 섹션에서는 ConversationalMessage 형식과 세션 기반 저장소를 사용합니다.**

In [ ]:
# MemorySession을 사용하여 이전 고객 상호작용으로 시드
previous_interactions = [
    ConversationalMessage("I bought a new iPhone 15 Pro on June 1st, 2025. Order number is 123456.", USER),
    ConversationalMessage("Thank you for your purchase! I can see your iPhone 15 Pro order #123456 was delivered successfully. How can I help you today?", ASSISTANT),
    ConversationalMessage("I also ordered Sennheiser headphones on June 20th. Order number 654321. They came with 1-year warranty.", USER),
    ConversationalMessage("Perfect! I have your Sennheiser headphones order #654321 on file with the 1-year warranty. Both your iPhone and headphones should work great together.", ASSISTANT),
    ConversationalMessage("I'm looking for a good laptop. I prefer ThinkPad models.", USER),
    ConversationalMessage("Great choice! ThinkPads are excellent for their durability and performance. Let me help you find the right model for your needs.", ASSISTANT)
]

# MemorySession을 사용하여 저장
try:
    event_response = customer_session.add_turns(previous_interactions)
    logger.info(f"✅ Seeded customer history using MemorySession")
    logger.info(f"   Event ID: {event_response['eventId']}")
except Exception as e:
    logger.error(f"⚠️ Error seeding history: {e}")


#### Agent가 준비되었습니다. 

### 고객 지원 시나리오를 테스트해 보겠습니다

In [ ]:
# 테스트 1: 고객이 iPhone 문제 보고
logger.info("🧪 Running Test 1: iPhone performance issue")
test_query_1 = "My iPhone is running very slow and gets hot when charging. Can you help?"
logger.info(f"Query: {test_query_1}")

response1 = support_agent(test_query_1)
logger.info(f"✅ Test 1 completed successfully")
print(f"\n📱 iPhone Issue Support Response:\n{response1}\n")

In [ ]:
# 테스트 2: Bluetooth 연결 문제
logger.info("🧪 Running Test 2: Bluetooth connectivity issue")
test_query_2 = "My iPhone won't connect to my Sennheiser headphones via Bluetooth. How do I fix this?"
logger.info(f"Query: {test_query_2}")

response2 = support_agent(test_query_2)
logger.info(f"✅ Test 2 completed successfully")
print(f"\n🎧 Bluetooth Issue Support Response:\n{response2}\n")

In [ ]:
# 테스트 3: 주문 상태 확인
logger.info("🧪 Running Test 3: Order status check")
test_query_3 = "Can you check the status of my recent orders?"
logger.info(f"Query: {test_query_3}")

response3 = support_agent(test_query_3)
logger.info(f"✅ Test 3 completed successfully")
print(f"\n📦 Order Status Support Response:\n{response3}\n")

In [ ]:
# 테스트 4: 선호도 기반 제품 추천
logger.info("🧪 Running Test 4: Product recommendation")
test_query_4 = "I'm still interested in buying a laptop. What ThinkPad models do you recommend?"
logger.info(f"Query: {test_query_4}")

response4 = support_agent(test_query_4)
logger.info(f"✅ Test 4 completed successfully")
print(f"\n💻 Product Recommendation Support Response:\n{response4}\n")

logger.info("🎉 All customer support scenario tests completed!")

#### 고객 지원 튜토리얼 완료! 🎉 
주요 내용:
- Memory hook은 MemorySessionManager를 사용하여 지원 세션 전반에 걸쳐 고객 컨텍스트를 자동으로 관리합니다
- 멀티 전략 메모리는 타입이 지정된 전략 클래스를 사용하여 대화에서 주문, 선호도 및 사실을 캡처합니다
- Agent는 고객 기록을 기반으로 개인화된 지원을 제공할 수 있습니다
- Tool은 주문 조회 및 웹 검색 기능을 위해 통합될 수 있습니다
- 지속적인 메모리로 고객 지원이 더욱 효율적이 됩니다

## 정리

### 선택 사항: Memory 리소스 삭제

In [ ]:
# MemoryManager를 사용하여 메모리 리소스 삭제하려면 주석 해제
# try:
#     memory_manager.delete_memory(memory_id)
#     print(f"✅ Deleted memory resource using MemoryManager: {memory_id}")
# except Exception as e:
#     print(f"Error deleting memory: {e}")